In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# （コア数128＆異なる問題サイズでの結果）OR（異なるコア数＆異なる問題サイズでの結果）

In [ ]:


resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}
date = "2021年11月17日"

resultDict128WithoutInit = {}
resultDict128WithInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}（コア数128））",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDict128WithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="128WithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="128WithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDict128WithoutInit).copy
)
#

# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
}

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    # print(
    #     dfBase.to_latex(
    #         caption=f"初期変数を含めずに作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}BaseMAPE{benchmarkName}",
    #     )
    # )

    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    # print(
    #     dfInit.to_latex(
    #         caption=f"初期変数を含めて作成したモデルのMAPE（ベンチマーク{benchmarkName.upper()}）",
    #         label=f"{date}InitMAPE{benchmarkName}",
    #     )
    # )
    #
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")

# （コア数128＆異なる問題サイズでの結果）OR（異なるコア数＆異なる問題サイズでの結果）

## 「コア数に線形モデル、問題サイズに反比例モデル」となる新モデルを追加

In [ ]:
resultsList = []

# コア数を128に固定し、問題サイズを振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [128]
dictColumnNamesJP = {
    "functionName": "関数名",
    "modelLin": "線形モデル",
    "modelIp": "反比例モデル",
    "modelLog": "対数モデル",
    "modelLinAndIp": "線形および反比例モデル",
}
model_names_list = list(dictColumnNamesJP.keys())
model_names_list.remove("functionName")
print(f"model_names_list={model_names_list}")
date = "2021年12月25日"

In [ ]:
# コア数と問題サイズの両方を振る

classes = ["A", "B", "C", "D", "E", "F"]
processes = [2, 4, 8, 16, 32, 64, 128, 256]

resultDictAllWithInit = {}
resultDictAllWithoutInit = {}

for benchmarkName in benchmarkNames:
    print(f"% _____benchmarkName={benchmarkName}_____")
    expVarBase = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=True,
        initExpVar=False,
    )
    dfBase = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarBase,
        csvDirPath=csvDirPath,
        modelNames=model_names_list,
    )
    dfBase = dfBase.rename(columns=dictColumnNamesJP)
    dfBase = dfBase.set_index("関数名")
    print(dfBase)
    #
    resultDictAllWithoutInit[benchmarkName.upper()] = addLowestMAPEColumn(dfBase)
    #
    expVarInit = returnExplanatoryVariablesList(
        benchmarkName=benchmarkName,
        classes=classes,
        processes=processes,
        csvDirPath=csvDirPath,
        baseExpVar=False,
        initExpVar=True,
    )
    dfInit = returnDictAboutMAPETable(
        benchmarkName=benchmarkName,
        classes=classes,
        targetClass=None,
        processes=processes,
        targetProcess=None,
        expVar=expVarInit,
        csvDirPath=csvDirPath,
    )
    dfInit = dfInit.rename(columns=dictColumnNamesJP)
    dfInit = dfInit.set_index("関数名")
    print(dfInit)
    resultDictAllWithInit[benchmarkName.upper()] = addLowestMAPEColumn(dfInit)
    #

csvDirPath = "./csv_files/"
test_returnExplanatoryVariablesList(csvDirPath=csvDirPath)

#
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).to_latex(
        caption=f"初期変数を含めて構築したモデルの平均MAPE", label="AllWithInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithInit).copy
)
print(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).to_latex(
        caption=f"初期変数を含めずに構築したモデルの平均MAPE", label="AllWithoutInit"
    )
)
resultsList.append(
    returnDFBenchmarkNameAndAverageLowestMAPE(resultDictAllWithoutInit).copy
)
# .to_latex(caption=f"初期変数を含めて作成したモデルの平均MAPE")